In [3]:
import re
import os
import json
import pandas as pd
from unittest import result 
import pprint
import pickle
import requests

In [7]:
input_dir = r'G:/goskat/'

In [ ]:
# Обходим директорию с набором данных госкаталога и выгружаем список всех вхождений типа "фотографии и негативы"
data_list = []
for dirpath, dirnames, filenames in os.walk(input_dir):
        for filepath in filenames:
            if filepath[-4:] == 'json':
                file = open( os.path.join(input_dir, filepath), "r", encoding = "utf8")
                b = json.load(file)
                for x in b:
                    try:
                        if x.get('data').get('typology').get('name') == 'фотографии и негативы':
                            data_list.append(x)
                    except:
                        pass
                file.close()


In [6]:
# Сохраняем дамп списка изображений и негативов отфильтрованных из набора данных госкаталога.
with open(r'G:/goskat/list1.pickle', 'wb') as f:
    pickle.dump(data_list, f)

In [3]:
# Загружаем дамп списка изображений и негативов отфильтрованных из набора данных госкаталога.
with open(r'G:/goskat/list.pickle', 'rb') as f:
    data_list = pickle.load(f)

In [ ]:
# Проверяем длину списка фотографий и негативов
len(data_list)

In [9]:
# Определяем функции поиска ключевого паттерна в полях "Автор", "Название", "Описание".
def true_auth(author_list,pattern):
    flag = False
    if author_list:
        for x in author_list:
            if re.search(pattern,x):
                flag = True
    return flag

def true_name(name,pattern):
    flag = False
    if name:
        if re.search(pattern,name):
            flag = True
    return flag

def true_desc(desc,pattern):
    flag = False
    if desc:
        if re.search(pattern,desc):
            flag = True
    return flag

In [9]:
# Создаем фрейм отфильтрованных по паттерну "Булл" данных изображений и негативов
df2 = pd.DataFrame(columns=['id','Название','Авторы','Тип','Описание','Провенанс','Место_происхождения','Музей','Размеры','Главные слова','Технологии','start_date','image_url'])  
pattern = r'(^|\s)Булл'
for x in data_list:
    item_name = x.get('data').get('name')
    item_description = x.get('data').get('description')
    item_authors= x.get('data').get('authors')
    
    if true_name(item_name,pattern) or true_desc(item_description,pattern) or true_auth(item_authors,pattern):

        image_url_list  = x.get('data').get('images')#ссылка на изображение
        image_url=[]
        if image_url_list:
            for u in image_url_list:
                image_url.append(u.get('url'))
        item_id =x.get('data').get('id')
        item_type = x.get('data').get('typology').get('name')
        
        item_production_place = x.get('data').get('productionPlace')
        item_museum = x.get('data').get('museum').get('name')
        item_dim = x.get('data').get('dimStr')
        item_technologies = x.get('data').get('technologies')
        item_start_date = x.get('data').get('startDate')
        item_authors= x.get('data').get('authors')
        item_provenance =   x.get('data').get('provenance')
        item_main_words = x.get('data').get('mainWords')


        item_data = {'id':item_id,'Название':item_name,'Авторы':item_authors,'Тип':item_type,'Описание':item_description,'Провенанс':item_provenance,'Место_происхождения':item_production_place,'Музей':item_museum,'Размеры':item_dim,'Главные слова':item_main_words,'Технологии':item_technologies,'start_date':item_start_date,'image_url':image_url}
        df1 = pd.DataFrame([item_data])
        # print(df1)
        df2 = pd.concat([df2,df1],axis = 0,ignore_index=True)

In [48]:
# Сохраняем дамп фрейма
with open(r'G:/goskatalog/df2.pickle', 'wb') as f:
    pickle.dump(df2, f)

In [ ]:
# Загружаем дамп фрейма
with open(r'G:/goskatalog/df2.pickle', 'rb') as f:
    df2 = pickle.load(f)

In [11]:
#Оцениваем объем фрейма
print(len(df2))

1470


In [10]:
# Проверяем содержание ссылок в слоте произвольной записи
pprint.pprint(df2['image_url'].loc[df2.index[11]])

['http://goskatalog.ru/muzfo-imaginator/rest/images/original/21821790',
 'http://goskatalog.ru/muzfo-imaginator/rest/images/original/21821795']


In [12]:
# Создаем счетчик вхождений имеющих более одного изображения)
counter =0
for n in range(len(df2)):
    if len(df2['image_url'].loc[df2.index[n-1]])>1:
        counter+=1
print(counter)
       

97


In [13]:
# Создаем список ID вхождений имеющих более одного изображения)
index_list =[]
for x in range(len(df2)):
    if len(df2['image_url'].loc[df2.index[x-1]])>1:
        # print('yes!')
        index_list.append(df2.index[x-1])

In [14]:
# Выводим содержание спика ID вхождений имеющих более одного изображения
print(index_list)

[0, 1, 3, 6, 20, 21, 57, 165, 233, 313, 314, 384, 385, 411, 424, 458, 464, 691, 692, 694, 695, 696, 697, 698, 700, 847, 854, 866, 973, 974, 975, 976, 977, 978, 979, 980, 981, 982, 983, 984, 985, 986, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034, 1035, 1036, 1037, 1038, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 1053, 1054, 1055, 1081, 1082, 1083, 1084, 1085, 1086, 1116]


In [15]:
# Создаем фрейм содержащий вхождения с не более чем одним изображением
df_one = df2.drop(index_list, axis = 0)
df_one.reset_index(drop= True , inplace= True )

In [16]:
# Выводим фрейм  содержащий вхождения с не более чем одним изображением
df_one

,id,Название,Авторы,Тип,Описание,Провенанс,Место_происхождения,Музей,Размеры,Главные слова,Технологии,start_date,image_url
2,21021483,"Фотография. Слушатели Курсов естествоведения, ...",[Булла Карл Карлович (Фотограф)],фотографии и негативы,None,None,"Российская империя, г. Санкт-Петербург",Бюджетное учреждение культуры Вологодской обла...,"21 х 28,8 см",None,"[картон, фотобумага черно-белая глянцевая, фот...",1913-01-01T12:00:00.000Z,[http://goskatalog.ru/muzfo-imaginator/rest/im...
4,21124358,Фотопортрет индивидуальный. Бочкарева (Арханге...,None,фотографии и негативы,Черно-белый снимок на фирменном картонном пасп...,None,"Российская империя, г. Санкт-Петербург","Краевое государственное бюджетное учреждение ""...","14x10 см; паспарту 16x10,6 см",None,"[Фотобумага, картон/ фотопечать]",None,[http://goskatalog.ru/muzfo-imaginator/rest/im...
5,21167092,Фотография черно-белая на почтовой карточке. Г...,"[Булла, братья (?)]",фотографии и негативы,"Оба стоя, в рост. Уланова слева, корпус анфас,...",None,СССР. Ленинград,Санкт-Петербургское государственное бюджетное ...,"13,8 х 8,8",None,"[фотопечать черно-белая, почтовая карточка]",1928-01-01T12:00:00.000Z,[http://goskatalog.ru/muzfo-imaginator/rest/im...
7,21589033,Фотография [Петров Г.],[Булла К. К.],фотографии и негативы,Фотоснимок сделан в профиль. Изображен [Петров...,None,г. Санкт-Петербург,Государственное бюджетное учреждение культуры ...,"верт. размер: 16,2, гориз. размер: 10,6, комм....",None,"[картон, фотопечать, печать типографская, тисн...",1880-01-01T12:00:00.000Z,[http://goskatalog.ru/muzfo-imaginator/rest/im...
8,21667275,"Фото ""Седов Георгий Яковлевич - российский гид...","[Фото ""К.К. Булла.С.П.Б. Невский проспект -54""]",фотографии и негативы,Фотография наклеена на паспарту . В нижней час...,None,С- Петербург,Государственное бюджетное учреждение культуры ...,"13,8 х 9,5 см,, 16,2 х 11,7 см",None,"[Фотобумага матовая, черно - белая, фотопечать...",None,[http://goskatalog.ru/muzfo-imaginator/rest/im...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170,20584925,"Крестьянские дети на фоне усадьбы ""Ясная Поляна"".",[Булла Карл Карлович (?)],фотографии и негативы,None,None,Ясная Поляна.,Федеральное государственное бюджетное учрежден...,"7,6х11,7",None,None,1908-01-01T12:00:00.000Z,[http://goskatalog.ru/muzfo-imaginator/rest/im...
1171,20685688,Афиша концерта Оле Булль (Ole Bull) в апреле 1...,None,фотографии и негативы,None,None,Христиана,Федеральное государственное бюджетное учрежден...,"10х8; 13х11,5",None,[фотография],1838-01-01T12:00:00.000Z,[http://goskatalog.ru/muzfo-imaginator/rest/im...
1172,20685988,Афиша концерта Оле Булль (Ole Bull) 6 апреля 1...,None,фотографии и негативы,None,None,Христиана,Федеральное государственное бюджетное учрежден...,"9х9,5",None,[фотография],1838-01-01T12:00:00.000Z,[http://goskatalog.ru/muzfo-imaginator/rest/im...
1173,20689170,Портрет Оле Булль (Ole Bull) со скрипкой в руках.,None,фотографии и негативы,Фотопортрет прикреплен к листу бумаги верхним ...,None,Христиана,Федеральное государственное бюджетное учрежден...,30х23,None,[фотография],1838-01-01T12:00:00.000Z,[http://goskatalog.ru/muzfo-imaginator/rest/im...


In [17]:
# Создаем фрейм содержащий вхождения с более чем одним изображением
df_double = df2.loc[df2.index[index_list]]
df_one.reset_index(drop= True , inplace= True )

In [18]:
# Выводим фрейм  содержащий вхождения с  более чем одним изображением
df_double

,id,Название,Авторы,Тип,Описание,Провенанс,Место_происхождения,Музей,Размеры,Главные слова,Технологии,start_date,image_url
0,20773912,Фотография Илья Артемьевич Бутин,[Фотограф: Булл К.К.],фотографии и негативы,На фотографии присутствует изображение человек...,None,None,"Государственное учреждение культуры ""Нерчински...","паспарту- 16,4 х 10,7 см., фотографии- 14 х 10...",None,"[фотобумага, картон, фотопечать, ч/б]",None,[http://goskatalog.ru/muzfo-imaginator/rest/im...
1,20803079,Фотография. Николай Константинович Рукавишников.,[Булла К. (Автор фотографии)],фотографии и негативы,None,None,"Российская империя, г. Санкт-Петербург",Бюджетное учреждение культуры Удмуртской Респу...,"8,9х5,9; 20,7х14,3 (с паспарту)",None,"[фотопечать чёрно-белая, фотобумага матовая, к...",1912-01-01T12:00:00.000Z,[http://goskatalog.ru/muzfo-imaginator/rest/im...
3,21108678,Фотография – Александр Дмитриевич Далматов и ...,None,фотографии и негативы,"Москва, Ходынское поле, фотограф К.К. Булла",None,None,"Муниципальное бюджетное учреждение ""Музей горо...","13,7 х 11,3 см",None,"[фотобумага, ч/б печать, штемпельная краска]",None,[http://goskatalog.ru/muzfo-imaginator/rest/im...
6,21502144,Фото сюжетное. Перенесение мощей Серафима Саро...,[Братья Булла],фотографии и негативы,Фото сюжетное. Перенесение мощей Серафима Саро...,None,г. Санкт-Петербург,Федеральное государственное бюджетное учрежден...,"16,5Х23",None,[бумага глянцевая],1901-01-01T12:00:00.000Z,[http://goskatalog.ru/muzfo-imaginator/rest/im...
20,22318445,Портрет Александра Трубникова.,[Фотограф К.К. Булла.],фотографии и негативы,None,None,Санкт-Петербург.,Санкт-Петербургское государственное бюджетное ...,"13,7 х 10,1; 16,3 х 10,5 см.",Фотография.,[фотобумага. фотопечать.],None,[http://goskatalog.ru/muzfo-imaginator/rest/im...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083,17691910,Россия. Лифляндская губерния. Рига. Торжествен...,"[Булла К. К., фотограф.]",фотографии и негативы,Вид с нижней точки на конную статую Петра I на...,None,"Россия. Рига, Санкт-Петербург.",Санкт-Петербургское государственное бюджетное ...,"29,3х23,6 см.",Фотография.,[Фотобумага. Фотопечать.],None,[http://goskatalog.ru/muzfo-imaginator/rest/im...
1084,17691938,"Россия. Лифляндская губерния. Рига, окрестност...","[Булла К. К., фотограф.]",фотографии и негативы,"Вид на стелу, увенчанную скульптурным изображе...",None,"Россия. Рига, Санкт-Петербург.",Санкт-Петербургское государственное бюджетное ...,"29,8х23,8 см.",Фотография.,[Фотобумага. Фотопечать.],None,[http://goskatalog.ru/muzfo-imaginator/rest/im...
1085,17691945,Россия. Ярославская губерния. Ростов Великий. ...,"[Булла К. К., фотограф.]",фотографии и негативы,Вид на храмы кремля с луковичными главками и с...,None,"Россия. Ростов, Санкт-Петербург.",Санкт-Петербургское государственное бюджетное ...,"18х23,5 см.",Фотография.,[Фотобумага. Фотопечать.],None,[http://goskatalog.ru/muzfo-imaginator/rest/im...
1086,17691956,Россия. Ярославская губерния. Ростов. Выход из...,"[Булла К. К., фотограф.]",фотографии и негативы,Николай II в окружении церковных чинов и свиты...,None,"Россия. Ростов, Санкт-Петербург.",Санкт-Петербургское государственное бюджетное ...,"17,9х23,7 см.",Фотография.,[Фотобумага. Фотопечать.],None,[http://goskatalog.ru/muzfo-imaginator/rest/im...


In [ ]:
# Скачивание набора изображений 
r = requests.get(url)
image_directory = r'G:/bulla/one/'

for x in range(len(df_one)):
    url_counter = 1
    url_dump = df_one['image_url'].loc[df_one.index[x-1]]
    img_name = df_one['id'].loc[df_one.index[x-1]]
    for url in url_dump:
        r = requests.get(url)
        with open(image_directory+ str(img_name)+'.'+ str(url_counter)+'.jpg', 'wb') as img:  
            img.write(r.content)
        url_counter+=1

In [25]:
# Загружаем изображения в директорию для вхождений с одним изображением и присваиваем файлу имя основанное на ID
image_directory = r'G:/bulla/one_2/'
for x in range(len(df_one)):
    url_counter = 1
    url_dump = df_one['image_url'].loc[df_one.index[x-1]]
    img_name = df_one['id'].loc[df_one.index[x-1]]
    for url in url_dump:
        r = requests.get(url)
        with open(image_directory+ str(img_name)+'.'+ str(url_counter)+'.jpg', 'wb') as img:  
            img.write(r.content)
        url_counter+=1

In [24]:
# Загружаем изображения в директорию для вхождений с более чем одним изображением и присваиваем файлам имя основанное на ID и номере изображения во вхождении

image_directory = r'G:/bulla/double_2/'
for x in range(len(df_double)):
    url_counter = 1
    url_dump = df_double['image_url'].loc[df_double.index[x-1]]
    img_name = df_double['id'].loc[df_double.index[x-1]]
    for url in url_dump:
        r = requests.get(url)
        with open(image_directory+ str(img_name)+'.'+ str(url_counter)+'.jpg', 'wb') as img:  
            img.write(r.content)
        url_counter+=1

In [27]:
# Проверяем закачанные файлы на пустоту. Создаем список вхождений с битыми или неактивными ссылками и список индексов вхождений. Для Вхождений с одним изображением
image_directory = r'G:/bulla/one_2/'
error_list=[]
error_index_list =[]
label=r'Название'
for dirpath, dirnames, filenames in os.walk(image_directory):
        for filepath in filenames:
            if os.path.getsize(os.path.join(image_directory,filepath)) <=0:
                file_id = filepath[:-6] 
                error_list.append(file_id)
                error_index_list.append(df_one.loc[df_one['id']== int(file_id)].index.item())

Молебен у Казанского собора.
['http://goskatalog.ru/muzfo-imaginator/rest/images/original/31809996']
10043515
--------------
Отношение Министерства Императорского Двора в Управление Собственного Его Величества дворца с автографом Генерала-лейтенанта Мосолова, с черной сургучной печатью; скреплено с семью фотографиями К. Буллы "Формы охранной команды Аничкова дворца"
['http://goskatalog.ru/muzfo-imaginator/rest/images/original/31809888']
10297925
--------------
Вейнберг Петр Исаевич (1831-1908). Фотографии из архива Лукашевич К.В.
['http://goskatalog.ru/muzfo-imaginator/rest/images/original/30675818']
10332345
--------------
Придел святого Александра Невского
['http://goskatalog.ru/muzfo-imaginator/rest/images/original/31810619']
10540116
--------------
Святые апостолы Петр и Павел
['http://goskatalog.ru/muzfo-imaginator/rest/images/original/31810607']
10540127
--------------
Икона в память 17 октября 1888 года
['http://goskatalog.ru/muzfo-imaginator/rest/images/original/31810707']
1054

In [38]:
# Удаляем пустые файлы
image_directory = r'G:/bulla/one_2/'
error_list=[]
error_index_list =[]
label=r'Название'
for dirpath, dirnames, filenames in os.walk(image_directory):
        for filepath in filenames:
            if os.path.getsize(os.path.join(image_directory,filepath)) <=0:
                os.remove(os.path.join(image_directory,filepath))

                # print(len(error_list))
                # print(file_id)
                # print (os.path.getsize(os.path.join(image_directory,filepath)))

In [24]:
# Проверяем закачанные файлы на пустоту. Создаем список вхождений с битыми или неактивными ссылками и список индексов вхождений. Для Вхождений с более чем одним изображением
image_directory = r'G:/bulla/double_2/'
error_list=[]
error_index_list =[]
label=r'Название'
for dirpath, dirnames, filenames in os.walk(image_directory):
        for filepath in filenames:
            if os.path.getsize(os.path.join(image_directory,filepath)) <=0:
                file_id = filepath[:-6] 
                error_list.append(file_id)
                error_index_list.append(df_double.loc[df_double['id']== int(file_id)].index.item())
               

In [26]:
# Удаляем пустые файлы
image_directory = r'G:/bulla/double_2/'
error_list=[]
error_index_list =[]
label=r'Название'
for dirpath, dirnames, filenames in os.walk(image_directory):
        for filepath in filenames:
            if os.path.getsize(os.path.join(image_directory,filepath)) <=0:
                os.remove(os.path.join(image_directory,filepath))
                # print(len(error_list))
                # print(file_id)
                print (os.path.getsize(os.path.join(image_directory,filepath)))

In [31]:
# Очищаем фрейм от вхождений без изображений



In [36]:
# Очищаем фрейм от вхождений без изображений
df_double_clear = df_double.drop(error_index_list, axis= 0)

In [14]:
df_one_clear

,id,Название,Авторы,Тип,Описание,Провенанс,Место_происхождения,Музей,Размеры,Главные слова,Технологии,start_date,image_url
0,21021483,"Фотография. Слушатели Курсов естествоведения, ...",[Булла Карл Карлович (Фотограф)],фотографии и негативы,None,None,"Российская империя, г. Санкт-Петербург",Бюджетное учреждение культуры Вологодской обла...,"21 х 28,8 см",None,"[картон, фотобумага черно-белая глянцевая, фот...",1913-01-01T12:00:00.000Z,[http://goskatalog.ru/muzfo-imaginator/rest/im...
1,21124358,Фотопортрет индивидуальный. Бочкарева (Арханге...,None,фотографии и негативы,Черно-белый снимок на фирменном картонном пасп...,None,"Российская империя, г. Санкт-Петербург","Краевое государственное бюджетное учреждение ""...","14x10 см; паспарту 16x10,6 см",None,"[Фотобумага, картон/ фотопечать]",None,[http://goskatalog.ru/muzfo-imaginator/rest/im...
2,21167092,Фотография черно-белая на почтовой карточке. Г...,"[Булла, братья (?)]",фотографии и негативы,"Оба стоя, в рост. Уланова слева, корпус анфас,...",None,СССР. Ленинград,Санкт-Петербургское государственное бюджетное ...,"13,8 х 8,8",None,"[фотопечать черно-белая, почтовая карточка]",1928-01-01T12:00:00.000Z,[http://goskatalog.ru/muzfo-imaginator/rest/im...
3,21589033,Фотография [Петров Г.],[Булла К. К.],фотографии и негативы,Фотоснимок сделан в профиль. Изображен [Петров...,None,г. Санкт-Петербург,Государственное бюджетное учреждение культуры ...,"верт. размер: 16,2, гориз. размер: 10,6, комм....",None,"[картон, фотопечать, печать типографская, тисн...",1880-01-01T12:00:00.000Z,[http://goskatalog.ru/muzfo-imaginator/rest/im...
4,21667275,"Фото ""Седов Георгий Яковлевич - российский гид...","[Фото ""К.К. Булла.С.П.Б. Невский проспект -54""]",фотографии и негативы,Фотография наклеена на паспарту . В нижней час...,None,С- Петербург,Государственное бюджетное учреждение культуры ...,"13,8 х 9,5 см,, 16,2 х 11,7 см",None,"[Фотобумага матовая, черно - белая, фотопечать...",None,[http://goskatalog.ru/muzfo-imaginator/rest/im...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1368,20584925,"Крестьянские дети на фоне усадьбы ""Ясная Поляна"".",[Булла Карл Карлович (?)],фотографии и негативы,None,None,Ясная Поляна.,Федеральное государственное бюджетное учрежден...,"7,6х11,7",None,None,1908-01-01T12:00:00.000Z,[http://goskatalog.ru/muzfo-imaginator/rest/im...
1369,20685688,Афиша концерта Оле Булль (Ole Bull) в апреле 1...,None,фотографии и негативы,None,None,Христиана,Федеральное государственное бюджетное учрежден...,"10х8; 13х11,5",None,[фотография],1838-01-01T12:00:00.000Z,[http://goskatalog.ru/muzfo-imaginator/rest/im...
1370,20685988,Афиша концерта Оле Булль (Ole Bull) 6 апреля 1...,None,фотографии и негативы,None,None,Христиана,Федеральное государственное бюджетное учрежден...,"9х9,5",None,[фотография],1838-01-01T12:00:00.000Z,[http://goskatalog.ru/muzfo-imaginator/rest/im...
1371,20689170,Портрет Оле Булль (Ole Bull) со скрипкой в руках.,None,фотографии и негативы,Фотопортрет прикреплен к листу бумаги верхним ...,None,Христиана,Федеральное государственное бюджетное учрежден...,30х23,None,[фотография],1838-01-01T12:00:00.000Z,[http://goskatalog.ru/muzfo-imaginator/rest/im...


In [13]:
df_double_clear

,id,Название,Авторы,Тип,Описание,Провенанс,Место_происхождения,Музей,Размеры,Главные слова,Технологии,start_date,image_url
0,20773912,Фотография Илья Артемьевич Бутин,[Фотограф: Булл К.К.],фотографии и негативы,На фотографии присутствует изображение человек...,None,None,"Государственное учреждение культуры ""Нерчински...","паспарту- 16,4 х 10,7 см., фотографии- 14 х 10...",None,"[фотобумага, картон, фотопечать, ч/б]",None,[http://goskatalog.ru/muzfo-imaginator/rest/im...
1,20803079,Фотография. Николай Константинович Рукавишников.,[Булла К. (Автор фотографии)],фотографии и негативы,None,None,"Российская империя, г. Санкт-Петербург",Бюджетное учреждение культуры Удмуртской Респу...,"8,9х5,9; 20,7х14,3 (с паспарту)",None,"[фотопечать чёрно-белая, фотобумага матовая, к...",1912-01-01T12:00:00.000Z,[http://goskatalog.ru/muzfo-imaginator/rest/im...
3,21108678,Фотография – Александр Дмитриевич Далматов и ...,None,фотографии и негативы,"Москва, Ходынское поле, фотограф К.К. Булла",None,None,"Муниципальное бюджетное учреждение ""Музей горо...","13,7 х 11,3 см",None,"[фотобумага, ч/б печать, штемпельная краска]",None,[http://goskatalog.ru/muzfo-imaginator/rest/im...
6,21502144,Фото сюжетное. Перенесение мощей Серафима Саро...,[Братья Булла],фотографии и негативы,Фото сюжетное. Перенесение мощей Серафима Саро...,None,г. Санкт-Петербург,Федеральное государственное бюджетное учрежден...,"16,5Х23",None,[бумага глянцевая],1901-01-01T12:00:00.000Z,[http://goskatalog.ru/muzfo-imaginator/rest/im...
20,22318445,Портрет Александра Трубникова.,[Фотограф К.К. Булла.],фотографии и негативы,None,None,Санкт-Петербург.,Санкт-Петербургское государственное бюджетное ...,"13,7 х 10,1; 16,3 х 10,5 см.",Фотография.,[фотобумага. фотопечать.],None,[http://goskatalog.ru/muzfo-imaginator/rest/im...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1378,17691910,Россия. Лифляндская губерния. Рига. Торжествен...,"[Булла К. К., фотограф.]",фотографии и негативы,Вид с нижней точки на конную статую Петра I на...,None,"Россия. Рига, Санкт-Петербург.",Санкт-Петербургское государственное бюджетное ...,"29,3х23,6 см.",Фотография.,[Фотобумага. Фотопечать.],None,[http://goskatalog.ru/muzfo-imaginator/rest/im...
1379,17691938,"Россия. Лифляндская губерния. Рига, окрестност...","[Булла К. К., фотограф.]",фотографии и негативы,"Вид на стелу, увенчанную скульптурным изображе...",None,"Россия. Рига, Санкт-Петербург.",Санкт-Петербургское государственное бюджетное ...,"29,8х23,8 см.",Фотография.,[Фотобумага. Фотопечать.],None,[http://goskatalog.ru/muzfo-imaginator/rest/im...
1380,17691945,Россия. Ярославская губерния. Ростов Великий. ...,"[Булла К. К., фотограф.]",фотографии и негативы,Вид на храмы кремля с луковичными главками и с...,None,"Россия. Ростов, Санкт-Петербург.",Санкт-Петербургское государственное бюджетное ...,"18х23,5 см.",Фотография.,[Фотобумага. Фотопечать.],None,[http://goskatalog.ru/muzfo-imaginator/rest/im...
1381,17691956,Россия. Ярославская губерния. Ростов. Выход из...,"[Булла К. К., фотограф.]",фотографии и негативы,Николай II в окружении церковных чинов и свиты...,None,"Россия. Ростов, Санкт-Петербург.",Санкт-Петербургское государственное бюджетное ...,"17,9х23,7 см.",Фотография.,[Фотобумага. Фотопечать.],None,[http://goskatalog.ru/muzfo-imaginator/rest/im...


In [40]:
# Сохраняем дамп фрейма
with open(r'G:/goskat/df_one_clear1.pickle', 'wb') as f:
    pickle.dump(df_one_clear, f)

In [41]:
# Сохраняем дамп фрейма
with open(r'G:/goskat/df_double_clear1.pickle', 'wb') as f:
    pickle.dump(df_double_clear, f)

In [4]:
# Загружаем дамп фрейма
with open(os.path.join(r'G:\bulla\main_data\df_goskatalog',r'df_one_clear.pickle'), 'rb') as f:
    df_one_clear = pickle.load(f)

In [5]:
# Загружаем дамп фрейма
with open(os.path.join(r'G:\bulla\main_data\df_goskatalog',r'G:/goskat/df_double_clear.pickle'), 'rb') as f:
    df_double_clear = pickle.load(f)

In [6]:
# Доступ к записи по ID
df_one_clear[df_one_clear['id']== 13146508] 

,id,Название,Авторы,Тип,Описание,Провенанс,Место_происхождения,Музей,Размеры,Главные слова,Технологии,start_date,image_url
951,13146508,Фотография,None,фотографии и негативы,"Буллякулова Б.-вдова, с.Султанмуратово",None,None,Султанмуратовский музей им. Г.Ибрагимова - фил...,длина 18 ширина 12,None,None,None,[http://goskatalog.ru/muzfo-imaginator/rest/im...


In [16]:
# доступ к полю по ID
interest_id = 13146508
interest_field = r'Музей'
df_one_clear[df_one_clear['id']== interest_id][interest_field].item()

'Султанмуратовский музей им. Г.Ибрагимова - филиал МАУКИ "Аургазинский историко-краеведченский музей" муниципального района Аургазинский район Республики Башкортостан'

In [26]:
# Проверяем запись ошибочного вхождения
df_one_clear[df_one_clear['id']== 24940155] 

,id,Название,Авторы,Тип,Описание,Провенанс,Место_происхождения,Музей,Размеры,Главные слова,Технологии,start_date,image_url
171,24940155,Фотография. Буллиев Василий Иванович.,None,фотографии и негативы,"Оригинал, ч/б, глянцевая. Анфас. На снимке пог...","На снимке Буллиев Василий Иванович, первый в р...","СССР, КССР, г. Олонец","Муниципальное бюджетное учреждение ""Олонецкий ...",18 х 24 см.,Фотография,"[фотобумага, печать]",2050-01-02T12:00:00.000Z,[http://goskatalog.ru/muzfo-imaginator/rest/im...


In [30]:
#  Проверяем запись ошибочного вхождения
df_one_clear[df_one_clear['id']== 27035120] 

,id,Название,Авторы,Тип,Описание,Провенанс,Место_происхождения,Музей,Размеры,Главные слова,Технологии,start_date,image_url
333,27035120,Фотонегатив. Буллиев С.М. - тракторист совхоза...,[Блейхман Евгений Аркадьевич],фотографии и негативы,None,None,Олонецкий р-н,"Бюджетное учреждение ""Национальный музей Респу...","2,4 х 3,6 см.",None,[фотопленка черно-белая],1963-01-01T12:00:00.000Z,[http://goskatalog.ru/muzfo-imaginator/rest/im...


In [18]:
# Проверяем запись ошибочного вхождения
df_one_clear[df_one_clear['id']== 23324153] 

,id,Название,Авторы,Тип,Описание,Провенанс,Место_происхождения,Музей,Размеры,Главные слова,Технологии,start_date,image_url
26,23324153,Фотография,None,фотографии и негативы,Фотография. Буллер Давид Давидович – слесарь П...,,,"Муниципальное бюджетное учреждение ""Краеведчес...","7,9х11,0 см",,"[фотобумага, фотопечать]",1980-01-01T12:00:00.000Z,[http://goskatalog.ru/muzfo-imaginator/rest/im...


In [46]:
# Проверяем запись ошибочного вхождения
df_one_clear[df_one_clear['id']== 27035138] 

,id,Название,Авторы,Тип,Описание,Провенанс,Место_происхождения,Музей,Размеры,Главные слова,Технологии,start_date,image_url
334,27035138,Фотонегатив. Механизированное звено Буллиева С...,[Блейхман Евгений Аркадьевич],фотографии и негативы,None,None,Олонецкий р-н,"Бюджетное учреждение ""Национальный музей Респу...","2,4 х 3,6 см.",None,[фотопленка черно-белая],1963-01-01T12:00:00.000Z,[http://goskatalog.ru/muzfo-imaginator/rest/im...


In [47]:
# Проверяем запись ошибочного вхождения
df_one_clear[df_one_clear['id']== 27035235] 

,id,Название,Авторы,Тип,Описание,Провенанс,Место_происхождения,Музей,Размеры,Главные слова,Технологии,start_date,image_url
335,27035235,Фотонегатив. Механизированное звено Буллиева С...,[Блейхман Евгений Аркадьевич],фотографии и негативы,None,None,Олонецкий р-н,"Бюджетное учреждение ""Национальный музей Респу...","2,4 х 3,6 см.",None,[фотопленка черно-белая],1963-01-01T12:00:00.000Z,[http://goskatalog.ru/muzfo-imaginator/rest/im...


In [44]:
# Проверяем запись ошибочного вхождения
interest_id = 27035138
interest_field = r'Название'
df_one_clear[df_one_clear['id']== interest_id][interest_field].item()

'Фотонегатив. Механизированное звено Буллиева С.М. на сенокосе. Слева направо: С.М. Буллиев, В.П. Риккиев, А.П. Буренков'

In [45]:
# Проверяем запись ошибочного вхождения
interest_id = 27035235
interest_field = r'Название'
df_one_clear[df_one_clear['id']== interest_id][interest_field].item()

'Фотонегатив. Механизированное звено Буллиева С.М. на сенокосе'

In [25]:
# Проверяем запись ошибочного вхождения
df_one_clear[df_one_clear['id']== 27324932] 

,id,Название,Авторы,Тип,Описание,Провенанс,Место_происхождения,Музей,Размеры,Главные слова,Технологии,start_date,image_url
344,27324932,"Фотография групповая Долгов, Булло, Недопекин,...",None,фотографии и негативы,"Портрет групповой, почти в полный рост, анфас....",None,Литва,Муниципальное бюджетное учреждение культуры Ру...,"11,0х8,0",None,[фотобумага глянцевая ч/б; фотопечать],None,[http://goskatalog.ru/muzfo-imaginator/rest/im...


In [8]:
#Очищаем фрейм от ошибочных вхождений
df_one_clear = df_one_clear.drop([951], axis= 0)
df_one_clear = df_one_clear.drop([171], axis= 0)
df_one_clear = df_one_clear.drop([333], axis= 0)
df_one_clear = df_one_clear.drop([26], axis= 0)
df_one_clear = df_one_clear.drop([344], axis= 0)
df_one_clear = df_one_clear.drop([334,335], axis= 0)

In [9]:
# Узнаем длину очищенных датафреймов
print(len(df_one_clear))
print(len(df_double_clear))

1096
90


In [10]:
# Сохраняем дамп фрейма
with open(r'G:/bulla/df_double_clear_finish.pickle', 'wb') as f:
    pickle.dump(df_double_clear, f)

In [11]:
# Сохраняем дамп фрейма
with open(r'G:/bulla/df_one_clear_finish.pickle', 'wb') as f:
    pickle.dump(df_one_clear, f)